In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 

# qmc used for Poisson Disk Sampling
from scipy.stats.qmc import PoissonDisk

# Group Attention Simulator for Immersive Rooms

This notebook describes an exploratory simulator that captures the attentional distribution in immersive rooms using a physically-based model of human agent movement. Although the simulator will be eventually incorporated in the Unity game engine, the mathematical definition of agent attention is best prototyped in isolation using NumPy. Therefore, to help clarifying thoughts, I will describe here in mathematical terms how agent attention is represented in the immersive room, and how the movement of agent group is imprinted as the room's attentional memory.

### Part 0: Background

The starting point of this attention simulator comes from the theory of proxemics, proposed in Edward T. Hall's [*The Hidden Dimension*](https://archive.org/details/hiddendimensionhall00hall/page/n9/mode/2up). He specified a series of distance-based measures that thresholds human social interactions. This has inspired the early simulation of user behavior in early physically-realistic virtual environment (see [Yan and Kalay, 2006](https://link.springer.com/chapter/10.1007/978-1-4020-5131-9_4)). 

For this simulator, I interpret the discrete distance thresholds in these works more liberally, opting to represent the attention of virtual agents in parametric representation with continuous distribution in space. 

### Part 1: Agent Representation

We consider the 2D geometries of agents in the room (seen from the top) as the basis of agent representation. The room has a pre-defined interaction boundary $\Omega$ constrained by its physical dimensions. Let $a = 1, 2, ..., A$ be the agents present in the immersive room. They each have positions $\mathbf{x}_a = (x_a, y_a)$ and orientations $\theta_a$. These parameters allow us to create two types of distributions in the room:

1) Proximity-based attentional weight, $w(x, y)$. This is a function of attentional probability based solely on how closely gathered agents are, as per Hall's original theory. For each agent $a$, this can be represented as a 2D Gaussian distribution in the room:

$$
\begin{equation}
    w_a(x, y) = \exp \left(-\frac{(x - x_a)^2 + (y - y_a)^2}{2\sigma_a^2} \right).
\end{equation}
$$

2) Orientation-based attentional weight, $w(r, \theta)$. This is a function of attentional probability based on both the agents' distances and their orientations, similar to Yan and Kalay's implementation of a virtual user's social space. (will derive this later) 

In [ ]:
def sample_agents(num_agents = 12):

    